In [72]:
import pandas as pd
from scipy.stats import (shapiro, 
                         normaltest, 
                         pearsonr, 
                         spearmanr, 
                         ttest_ind)

from statsmodels.stats.weightstats import ztest

In [73]:
# набор данных

petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

pmean = pd.Series(data=petersburg).mean().round(4) # выравниваем количество элементов

petersburg.append(pmean)

In [74]:
# объединение данных в DataFrame

d = {'pet': petersburg, 'mag': magadan}

shells_df = pd.DataFrame(data=d)

shells_df

,pet,mag
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,0.1034,0.0689


In [75]:
# проверка данных на нормальность и корреляцию

h0 = 'Данные распределены нормально'
h1 = 'Данные не распределены нормально'

alpha = 0.05

def testnormcorr(data: pd.DataFrame, 
                 alpha: float):
    
      _, p = shapiro(data)
      
      if p > alpha:
            print(f'{p} > {alpha} \n{h0} \n'
                  '\n'
                  'Так что используем корреляцию Пирсона')
            
            c, pv = pearsonr(x=data.iloc[:, 0],
                             y=data.iloc[:, 1])
            
      else:
            print(f'{p} <= {alpha} \n{h1} \n'
                  '\n'
                  'Так что используем корреляцию Спирмена')
            
            c, pv = spearmanr(a=data.iloc[:, 0],
                              b=data.iloc[:, 1])
      
      print(f'Она равна {c}')
            

testnormcorr(shells_df, alpha)

0.10020463168621063 > 0.05 
Данные распределены нормально 

Так что используем корреляцию Пирсона
Она равна 0.2448446105155939


Вывод: данные имеют нормальное распределение и очень слабую положительную связь. Вероятно потому что мидий подбирали в случайном порядке.

In [77]:
# Какова разница в среднем размере мидии в зависимости от города-производителя?

h0 = 'Нет разницы между размерами мидий от разных производителей.'
h1 = 'Мидии из разных городов имеют различие в размерах.'

alpha = 0.05

def hypotest(data: pd.DataFrame, 
             alpha: float):
    
    groups = data.shape[1]
    objs = data.shape[0]
    
    if groups == 2:
        print(f'Мы сравниваем {groups} группы по {objs} объектов')
        # заранее известно, что это разные совокупности
        
        if objs < 30:
            print('Выборка составляет меньше 30 объектов \n'
                  'Следовательно используем независимый Т-тест \n')
            
            tt, p = ttest_ind(data.iloc[:, 0], 
                              data.iloc[:, 1], 
                              equal_var=True)
            
        else:
            print('Выборка составляет 30 или более объектов \n'
                  'Следовательно используем Z-тест \n')
            
            z, p = ztest(data.iloc[:, 0], 
                         data.iloc[:, 1], 
                         value=0, 
                         alternative='two-sided')

        
    if p > alpha:
        print(f'{p} > {alpha} \n' 
              f'Мы не можем отвергнуть нулевую гипотезу. {h0}')
    else:
        print(f'{p} <= {alpha} \n' 
              f'Мы отвергаем нулевую гипотезу. {h1}')
        
        
hypotest(shells_df, alpha)

Мы сравниваем 2 группы по 8 объектов
Выборка составляет меньше 30 объектов 
Следовательно используем независимый Т-тест 

0.0027388614510137294 <= 0.05 
Мы отвергаем нулевую гипотезу. Мидии из разных городов имеют различие в размерах.


Вывод: размеры раковин мидий из Балтийского и Охотского морей имеют значительную разницу. На мой взгляд на это влияет в первую очередь температура воды. Так как Балтийское море более теплое, чем Охотское, то мидии из Ленинграда будут больше, чем из Магадана.